## RFInder: tools to investigate the presence of RFI in a dataset


In [1]:
import os, string, sys
import numpy as np
import pyaml, json
from astropy.io import ascii

from IPython.display import HTML, display

import rfinder as rfinder

print '\t+------+\n\t  Done \n\t+------+'


Bad key "backed" on line 1 in
/Users/maccagni/.matplotlib/matplotlibrc.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution


	+------+
	  Done 
	+------+


### Load and modify parameter file

In [2]:
print '\t+------+\n\t RFInder \n\t Parameter File \n\t+------+'

rfi_par = rfinder.rfinder('/Users/maccagni/notebooks/rfinder/rfinder_default.yml')

print '\t+------+\n\t Parameters Loaded \n\t+------+'

#print parameter file 
print(json.dumps(rfi_par.cfg_par, indent=4, sort_keys=True))


	+------+
	 RFInder 
	 Parameter File 
	+------+
	+------+
	 Parameters Loaded 
	+------+
{
    "beam_shape": {
        "__helpstr": "estimate beam shape after RFI flagging", 
        "enable": false
    }, 
    "general": {
        "__helpstr": "General INPUT directories and filenames", 
        "msfullpath": "/Users/maccagni/Projects/FornaX/ATCA/WSRTA110311_B000.MS", 
        "msname": [
            "WSRTA110311_B000.MS"
        ], 
        "plotdir": "/Users/maccagni/Projects/FornaX/ATCA/rfi/plots/", 
        "rfidir": "/Users/maccagni/Projects/FornaX/ATCA/rfi/", 
        "tabledir": "/Users/maccagni/Projects/FornaX/ATCA/rfi/tables/", 
        "verbose": false, 
        "workdir": "/Users/maccagni/Projects/FornaX/ATCA/"
    }, 
    "plots": {
        "__helpstr": "plot rfi over different axes", 
        "enable": false, 
        "plot_long_short": true, 
        "plot_noise": "flag"
    }, 
    "rfi": {
        "__helpstr": "Find RFI in msfile over threshold", 
        "bad_antenna"

In [3]:
# Set working directory & filenames
rfi_par.cfg_par['general']['workdir'] = '/Users/maccagni/notebooks/rfinder/examples/'

# Enable / disable modules
rfi_par.cfg_par['rfi']['enable'] = True
rfi_par.cfg_par['beam_shape']['enable'] = True
rfi_par.cfg_par['plots']['enable'] = True

#MSFILE
rfi_par.cfg_par['general']['msname'] = ['WSRTA090807_B000.MS']



rfi_par.set_dirs()
# Options

print '\t+------+\n\t Parameters modified \n\t+------+'


	+------+
	 Parameters modified 
	+------+


### Investigate presence of RFI in a dataset

In [4]:
import rfi
import rfinder_plots as rfi_pl

rfi = rfi.rfi()

#read info from MSfile 
rfi.load_from_ms(rfi_par.cfg_par,0)
#sort visibilities by baseline lenght
rfi.baselines_from_ms(rfi_par.cfg_par)
#flag bad antennas (from configuration file)
datas = rfi.priors_flag(rfi_par.cfg_par)
#find rfi above threshold
rfi.find_rfi(datas,rfi_par.cfg_par,-1)
rfi_pl.rfi_frequency(rfi_par.cfg_par,-1)


print '\t+------+\n\t Done \n\t+------+'


Successful readonly open of default-locked table /Users/maccagni/notebooks/rfinder/examples/WSRTA090807_B000.MS/ANTENNA: 8 columns, 5 rows
	Total number of antennas	:5
	Antenna names	[0 1 2 3 4]
Successful readonly open of default-locked table /Users/maccagni/notebooks/rfinder/examples/WSRTA090807_B000.MS/SPECTRAL_WINDOW: 14 columns, 1 rows
	Bandwidth [kHz]	:12.20703125
	Start Frequency [GHz]	:1.30703125
	End Frequency [GHz]	:1.44139404297
Successful readonly open of default-locked table /Users/maccagni/notebooks/rfinder/examples/WSRTA090807_B000.MS: 22 columns, 8445 rows
... info from MS loaded ...

[([3, 4], 36.003190691114654), ([2, 3], 143.99243003098971), ([2, 4], 179.99562021086842), ([1, 2], 287.98137006111148), ([0, 1], 287.98287001667376), ([1, 3], 431.9738000409501), ([1, 4], 467.97699017946638), ([0, 2], 575.96424007024541), ([0, 3], 719.95667005550683), ([0, 4], 755.95986016610448)]
... baselines loaded...

... prior flagging done...

... RFI found...

	+------+
	 Done 
	+-

### Plot RFI

In [ ]:
import rfinder_plots as rfi_pl

#plot spectra
rfi_pl.plot_rfi_im(rfi_par.cfg_par,-1)
rfi_pl.rfi_frequency(rfi_par.cfg_par,-1)
rfi_pl.plot_noise_frequency(rfi_par.cfg_par,-1)

rfi_par.cfg_par['plots']['long_short'] = True
rfi_par.cfg_par['plots']['plot_noise'] = 'noise'

rfi_pl.plot_noise_frequency(rfi_par.cfg_par,-1)


print '\t+------+\n\t Plots done \n\t+------+'


... RFI in 2D plotted ----
... RFI in 1D plotted ----
... RFI in 1D plotted ----
	+------+
	 Plots done 
	+------+


### Determine RFI in data every time step store it in .fits tables

In [ ]:
import rfi
import rfinder_plots as rfi_pl

rfi = rfi.rfi()

rfi_par.cfg_par['time_chunks']['enable'] = True
rfi_par.cfg_par['time_chunks']['time_step'] = 10 #in minutes


#read info from MSfile 
#rfi.load_from_ms(rfi_par.cfg_par)
times = rfi.time_chunk(rfi_par.cfg_par)
for i in xrange(0,len(times)-1):
    
    timez = [times[i],times[i+1]] 
    rfi.load_from_ms(rfi_par.cfg_par,timez)
    #sort visibilities by baseline lenght
    rfi.baselines_from_ms(rfi_par.cfg_par)

    #flag bad antennas (from configuration file)
    datas = rfi.priors_flag(rfi_par.cfg_par)

    #find rfi above threshold
    rfi.find_rfi(datas,rfi_par.cfg_par,i)
    rfi_pl.rfi_frequency(rfi_par.cfg_par,i)

#create FLAG column in copy of MS file
#rfi.rfi_flag(rfi_par.cfg_par)

print '\t+------+\n\t Done \n\t+------+'

Successful readonly open of default-locked table /Users/maccagni/notebooks/rfinder/examples/WSRTA090807_B000.MS: 22 columns, 8445 rows
{'readme': 'This is a MeasurementSet Table holding measurements from a Telescope\n', 'subType': '', 'type': 'Measurement Set'}
5011749018.36
Start date: 17Sep10:03:40:18
End date: 17Sep10:08:28:03
Successful readonly open of default-locked table /Users/maccagni/notebooks/rfinder/examples/WSRTA090807_B000.MS/ANTENNA: 8 columns, 5 rows
	Total number of antennas	:5
	Antenna names	[0 1 2 3 4]
Successful readonly open of default-locked table /Users/maccagni/notebooks/rfinder/examples/WSRTA090807_B000.MS/SPECTRAL_WINDOW: 14 columns, 1 rows
	Bandwidth [kHz]	:12.20703125
	Start Frequency [GHz]	:1.30703125
	End Frequency [GHz]	:1.44139404297
Successful readonly open of default-locked table /Users/maccagni/notebooks/rfinder/examples/WSRTA090807_B000.MS: 22 columns, 8445 rows
... info from MS loaded ...

[([3, 4], 36.003190691114654), ([2, 3], 143.99243003098971),

### Determine shape of PSF after RFI flagging

In [ ]:
import rfinder_beam as rfi_beam

#create FLAG column in copy of MS file
rfi.rfi_flag(rfi_par.cfg_par)
beam = rfi_beam.make_psf(rfi_par.cfg_par)

print '\t+------+\n\t Spectra extracted \n\t+------+'
